In [1]:
import os
import torch

torch.cuda.set_device(1)

/home/test/anaconda3/envs/ldm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from omegaconf import OmegaConf
from ldm.util import instantiate_from_config
import torch
import json
import torch.nn.functional as F

In [7]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
inference_file = "./ProSpect_Classifier/data/CUB'/novel.json"       ### what inference file we are using 

### Loading the model to the config 
def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model.to(device)
    model.eval()
    return model






config="../../configs/stable-diffusion/v1-inference.yaml"
ckpt="../../models/sd/sd-v1-4.ckpt"
config = OmegaConf.load(f"{config}")
model = load_model_from_config(config, f"{ckpt}")
model = model.to(device)

Loading model from ../../models/sd/sd-v1-4.ckpt
Global Step: 470000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
make_beta_schedule:linear
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.16.self_attn.v_proj.weight', 'vision_model.encoder.layers.22.self_attn.k_proj.weight', 'vision_model.encoder.layers.15.mlp.fc1.weight', 'vision_model.encoder.layers.3.layer_norm2.weight', 'vision_model.encoder.layers.11.layer_norm1.bias', 'vision_model.encoder.layers.9.self_attn.out_proj.weight', 'vision_model.encoder.layers.18.self_attn.out_proj.weight', 'vision_model.encoder.layers.23.layer_norm1.bias', 'vision_model.encoder.layers.8.mlp.fc1.weight', 'vision_model.encoder.layers.3.mlp.fc1.bias', 'vision_model.encoder.layers.10.layer_norm1.bias', 'vision_model.encoder.layers.4.mlp.fc2.bias', 'vision_model.encoder.layers.5.mlp.fc1.weight', 'vision_model.encoder.layers.12.self_attn.k_proj.bias', 'vision_model.encoder.layers.20.layer_norm1.bias', 'vision_model.encoder.layers.1.layer_norm1.weight', 'vision_model.encoder.layers.11.mlp.fc1.bias',

In [23]:
condition = torch.load("../../ProSpect_Classifier/logs/class_embeddings_l2_21.pth")

In [24]:
condition_list_0 = []
for i in condition:
    condition_list_0.append(condition[i])
    
condition_list=torch.stack(condition_list_0,dim=0)
condition_list=condition_list.transpose(0,1)
condition_list.shape



temp=condition_list.unsqueeze(2).repeat(1,1,7,1,1)
x= F.cosine_similarity(temp,temp.transpose(1,2),dim=-1)
x=x.mean(dim=-1)


In [25]:
x

tensor([[[1.0000, 0.2169, 0.2944, 0.1946, 0.2718, 0.2641, 0.2399],
         [0.2169, 1.0000, 0.2441, 0.3328, 0.4282, 0.2920, 0.2742],
         [0.2944, 0.2441, 1.0000, 0.3478, 0.3342, 0.2168, 0.1938],
         [0.1946, 0.3328, 0.3478, 1.0000, 0.3746, 0.3116, 0.2132],
         [0.2718, 0.4282, 0.3342, 0.3746, 1.0000, 0.2903, 0.1650],
         [0.2641, 0.2920, 0.2168, 0.3116, 0.2903, 1.0000, 0.3133],
         [0.2399, 0.2742, 0.1938, 0.2132, 0.1650, 0.3133, 1.0000]],

        [[1.0000, 0.2295, 0.3684, 0.4005, 0.3388, 0.2780, 0.2730],
         [0.2295, 1.0000, 0.2911, 0.2231, 0.2595, 0.3335, 0.3165],
         [0.3684, 0.2911, 1.0000, 0.4309, 0.3502, 0.3969, 0.3884],
         [0.4005, 0.2231, 0.4309, 1.0000, 0.3631, 0.3123, 0.3795],
         [0.3388, 0.2595, 0.3502, 0.3631, 1.0000, 0.2900, 0.2220],
         [0.2780, 0.3335, 0.3969, 0.3123, 0.2900, 1.0000, 0.4899],
         [0.2730, 0.3165, 0.3884, 0.3795, 0.2220, 0.4899, 1.0000]],

        [[1.0000, 0.3815, 0.2447, 0.2722, 0.3850, 0.1746, 

: 

In [16]:
x

tensor([[[ 1.0000,  0.2169,  0.2944,  0.1946,  0.2718,  0.2641,  0.2399],
         [ 0.2169,  1.0000,  0.2441,  0.3328,  0.4282,  0.2920,  0.2742],
         [ 0.2944,  0.2441,  1.0000,  0.3478,  0.3342,  0.2168,  0.1938],
         [ 0.1946,  0.3328,  0.3478,  1.0000,  0.3746,  0.3116,  0.2132],
         [ 0.2718,  0.4282,  0.3342,  0.3746,  1.0000,  0.2903,  0.1650],
         [ 0.2641,  0.2920,  0.2168,  0.3116,  0.2903,  1.0000,  0.3133],
         [ 0.2399,  0.2742,  0.1938,  0.2132,  0.1650,  0.3133,  1.0000]],

        [[ 1.0000, -0.2257, -0.0299, -0.1858, -0.0837, -0.1312, -0.1989],
         [-0.2257,  1.0000,  0.0049, -0.1702, -0.1622, -0.0238, -0.1469],
         [-0.0299,  0.0049,  1.0000, -0.1058,  0.0104, -0.1170, -0.0866],
         [-0.1858, -0.1702, -0.1058,  1.0000, -0.1078, -0.2339, -0.2198],
         [-0.0837, -0.1622,  0.0104, -0.1078,  1.0000, -0.1084, -0.1774],
         [-0.1312, -0.0238, -0.1170, -0.2339, -0.1084,  1.0000, -0.1002],
         [-0.1989, -0.1469, -0.0866,